# Phase Extraction Demo
# Demonstrates ϕ(t) and χ(t) extraction from biosignals

This notebook demonstrates the triadic embedding:
$$\psi(t) = t + i\phi(t) + j\chi(t), \quad \chi(t) := \partial_t\phi(t)$$

where ϕ(t) is instantaneous phase and χ(t) captures phase acceleration/torsion.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.ndimage import gaussian_filter1d

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Hilbert Transform and Instantaneous Phase

For a real signal x(t), the analytic signal is:
$$z(t) = x(t) + i\mathcal{H}[x(t)]$$

The instantaneous phase is:
$$\phi(t) = \arg(z(t))$$

In [ ]:
def extract_phase(signal, unwrap=True):
    """
    Extract instantaneous phase via Hilbert transform.
    
    Parameters
    ----------
    signal : array_like
        Input time series
    unwrap : bool
        Whether to unwrap phase (remove 2π discontinuities)
    
    Returns
    -------
    phi : ndarray
        Instantaneous phase ϕ(t)
    """
    analytic_signal = hilbert(signal)
    phi = np.angle(analytic_signal)
    
    if unwrap:
        phi = np.unwrap(phi)
    
    return phi

## 2. Phase Derivative (Torsion)

The phase derivative χ(t) is computed with robust differentiation:
$$\chi(t) = \frac{d\phi}{dt}$$

In [ ]:
def compute_phase_derivative(phi, dt=1.0, sigma=2.0):
    """
    Compute phase derivative with Gaussian smoothing.
    
    Parameters
    ----------
    phi : array_like
        Instantaneous phase
    dt : float
        Time step (inverse of sampling rate)
    sigma : float
        Gaussian smoothing parameter for derivative
    
    Returns
    -------
    chi : ndarray
        Phase derivative χ(t)
    """
    # Smooth phase before differentiation
    phi_smooth = gaussian_filter1d(phi, sigma=sigma)
    
    # Compute derivative
    chi = np.gradient(phi_smooth, dt)
    
    return chi

## 3. Synthetic Signal Example

In [ ]:
# Generate synthetic signal with regime change
fs = 250  # Sampling rate (Hz)
duration = 10  # seconds
t = np.linspace(0, duration, fs * duration)

# Signal with frequency transition at t=5s
f1, f2 = 10, 25  # Hz
signal = np.where(
    t < 5,
    np.sin(2 * np.pi * f1 * t),
    np.sin(2 * np.pi * f2 * t)
)

# Add noise
signal += 0.1 * np.random.randn(len(signal))

# Extract phase and derivative
phi = extract_phase(signal)
chi = compute_phase_derivative(phi, dt=1/fs)

In [ ]:
# Visualization
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

# Original signal
axes[0].plot(t, signal, linewidth=0.8)
axes[0].axvline(5, color='red', linestyle='--', alpha=0.5, label='Regime change')
axes[0].set_ylabel('x(t)')
axes[0].set_title('Original Signal')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Instantaneous phase
axes[1].plot(t, phi, linewidth=0.8, color='blue')
axes[1].axvline(5, color='red', linestyle='--', alpha=0.5)
axes[1].set_ylabel('ϕ(t) [rad]')
axes[1].set_title('Instantaneous Phase')
axes[1].grid(True, alpha=0.3)

# Phase derivative
axes[2].plot(t, chi, linewidth=0.8, color='green')
axes[2].axvline(5, color='red', linestyle='--', alpha=0.5)
axes[2].set_ylabel('χ(t) [rad/s]')
axes[2].set_xlabel('Time (s)')
axes[2].set_title('Phase Derivative (Torsion)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean χ(t) before transition: {chi[t < 5].mean():.2f} rad/s")
print(f"Mean χ(t) after transition: {chi[t >= 5].mean():.2f} rad/s")

## 4. EEG Application Example

In [ ]:
# Simulate EEG-like signal (alpha band transition)
def simulate_eeg(duration=20, fs=250, transition_time=10):
    """
    Simulate EEG signal with alpha band power transition.
    """
    t = np.linspace(0, duration, fs * duration)
    
    # Alpha band (8-13 Hz) with amplitude modulation
    alpha_freq = 10
    alpha_amp = np.where(t < transition_time, 1.0, 0.3)
    alpha = alpha_amp * np.sin(2 * np.pi * alpha_freq * t)
    
    # Beta band (13-30 Hz) with inverse modulation
    beta_freq = 20
    beta_amp = np.where(t < transition_time, 0.3, 1.0)
    beta = beta_amp * np.sin(2 * np.pi * beta_freq * t)
    
    # Combine and add noise
    eeg = alpha + beta + 0.2 * np.random.randn(len(t))
    
    return t, eeg

# Generate and process
t_eeg, eeg_signal = simulate_eeg()
phi_eeg = extract_phase(eeg_signal)
chi_eeg = compute_phase_derivative(phi_eeg, dt=1/250)

In [ ]:
# Visualize EEG triadic embedding
fig, axes = plt.subplots(3, 1, figsize=(14, 8), sharex=True)

axes[0].plot(t_eeg, eeg_signal, linewidth=0.6, alpha=0.8)
axes[0].axvline(10, color='red', linestyle='--', alpha=0.5, label='Alpha→Beta transition')
axes[0].set_ylabel('EEG (μV)')
axes[0].set_title('Simulated EEG Signal')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(t_eeg, phi_eeg, linewidth=0.8, color='purple')
axes[1].axvline(10, color='red', linestyle='--', alpha=0.5)
axes[1].set_ylabel('ϕ_B(t) [rad]')
axes[1].set_title('Brain Phase')
axes[1].grid(True, alpha=0.3)

axes[2].plot(t_eeg, chi_eeg, linewidth=0.8, color='orange')
axes[2].axvline(10, color='red', linestyle='--', alpha=0.5)
axes[2].set_ylabel('χ_B(t) [rad/s]')
axes[2].set_xlabel('Time (s)')
axes[2].set_title('Brain Phase Torsion (Regime Marker)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. ECG/HRV Application Example

In [ ]:
# Simulate HRV time series
def simulate_hrv(duration=60, baseline_hr=70, transition_time=30):
    """
    Simulate heart rate variability signal.
    """
    t = np.linspace(0, duration, 4 * duration)  # 4 Hz sampling for HRV
    
    # Baseline heart rate with LF/HF components
    lf = 0.1  # Low frequency ~0.1 Hz
    hf = 0.25  # High frequency ~0.25 Hz
    
    # Shift LF/HF balance at transition
    lf_amp = np.where(t < transition_time, 5, 10)
    hf_amp = np.where(t < transition_time, 10, 5)
    
    hrv = baseline_hr + lf_amp * np.sin(2 * np.pi * lf * t) + \
          hf_amp * np.sin(2 * np.pi * hf * t) + \
          2 * np.random.randn(len(t))
    
    return t, hrv

t_hrv, hrv_signal = simulate_hrv()
phi_hrv = extract_phase(hrv_signal)
chi_hrv = compute_phase_derivative(phi_hrv, dt=0.25)

In [ ]:
# Visualize HRV triadic embedding
fig, axes = plt.subplots(3, 1, figsize=(14, 8), sharex=True)

axes[0].plot(t_hrv, hrv_signal, linewidth=0.8)
axes[0].axvline(30, color='red', linestyle='--', alpha=0.5, label='Autonomic shift')
axes[0].set_ylabel('HR (bpm)')
axes[0].set_title('Simulated HRV Signal')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(t_hrv, phi_hrv, linewidth=0.8, color='teal')
axes[1].axvline(30, color='red', linestyle='--', alpha=0.5)
axes[1].set_ylabel('ϕ_H(t) [rad]')
axes[1].set_title('Heart Phase')
axes[1].grid(True, alpha=0.3)

axes[2].plot(t_hrv, chi_hrv, linewidth=0.8, color='crimson')
axes[2].axvline(30, color='red', linestyle='--', alpha=0.5)
axes[2].set_ylabel('χ_H(t) [rad/s]')
axes[2].set_xlabel('Time (s)')
axes[2].set_title('Heart Phase Torsion')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. **Phase extraction** via Hilbert transform: ϕ(t) = arg(H[x(t)])
2. **Phase derivative** (torsion): χ(t) = ∂ϕ/∂t as a regime-change marker
3. **Application to EEG**: tracking alpha/beta transitions
4. **Application to HRV**: tracking autonomic balance shifts

The triadic embedding ψ(t) = (t, ϕ, χ) provides a sensitive framework for detecting nonstationary regime transitions in biosignals.

**Next steps**: See `02_feature_computation.ipynb` for ΔS, ΔI, ΔC calculation.